# Load Packages

In [1]:
#### Load Packages

import pandas as pd 
import numpy as np
from itertools import repeat
import math

# Raw Data Input & Data Manipulation -> household.dta

In [3]:
#### Import Raw Data from household.dta file

df = pd.read_stata('household.dta')
feature_names = df.columns
df_shape = df.shape
number_households_df = len(df.ssuid.unique())

#################################################################################################################


#### Data Manipulation

# Reference month transformation
def srefmon_numerical (row):
   if row['srefmon'] == 'First Reference month' :
      return 1
   if row['srefmon'] == 'Second Reference month' :
      return 2
   if row['srefmon'] == 'Third Reference month' :
      return 3
   if row['srefmon'] == 'Fourth Reference month' :
      return 4
   return 'Other'

df['srefmon_numerical'] = df.apply (lambda row: srefmon_numerical(row), axis=1)


# Observation month transformation
def month_numerical (row): 
    if row['month'] == 'January': 
        return 1
    if row['month'] == 'February': 
        return 2
    if row['month'] == 'March': 
        return 3
    if row['month'] == 'April': 
        return 4    
    if row['month'] == 'May': 
        return 5
    if row['month'] == 'June': 
        return 6
    if row['month'] == 'July': 
        return 7
    if row['month'] == 'August': 
        return 8
    if row['month'] == 'September': 
        return 9
    if row['month'] == 'October': 
        return 10
    if row['month'] == 'November': 
        return 11
    if row['month'] == 'December': 
        return 12

df['month_numerical'] = df.apply (lambda row: month_numerical(row), axis=1)


# Month of rebate receipt
def reb_month (row):
   if row['reb4'] == 1 :
      return 'April'
   if row['reb5'] == 1 :
      return 'May'
   if row['reb6'] == 1 :
      return 'June'
   if row['reb7'] == 1 :
      return 'July'
   if row['reb8'] == 1 :
      return 'August'
   if row['reb9'] == 1 :
      return 'September'
   if row['reb10'] == 1 :
      return 'October'
   if row['reb11'] == 1 :
      return 'November'
   if row['reb12'] == 1 :
      return 'December'
   return 'No rebate'

df['reb_month'] = df.apply (lambda row: reb_month(row), axis=1)



def reb_month_numerical (row):
    if row['reb4'] == 1 :
      return 4
    if row['reb5'] == 1 :
      return 5
    if row['reb6'] == 1 :
      return 6
    if row['reb7'] == 1 :
      return 7
    if row['reb8'] == 1 :
      return 8
    if row['reb9'] == 1 :
      return 9
    if row['reb10'] == 1 :
      return 10
    if row['reb11'] == 1 :
      return 11
    if row['reb12'] == 1 :
      return 12
    return 0 

df['reb_month_numerical'] = df.apply (lambda row: reb_month_numerical(row), axis=1)


# Rebate in this month
def rebate_payment (row):
   if row['reb_month'] == row['month'] :
      return 1
   return 0

df['rebate_payment'] = df.apply (lambda row: rebate_payment(row), axis=1)


# Dummy creation for rebate receipt
df['reb_receipt'] = df.reb4 + df.reb5 + df.reb6 + df.reb7 + df.reb8 + df.reb9 + df.reb10 + df.reb11 + df.reb12
df['reb_size'] = df.amt4 + df.amt5 + df.amt6 + df.amt7 + df.amt8 + df.amt9 + df.amt10 + df.amt11 + df.amt12


df['earn_male'] = df['earn_male'].fillna(0)
df['earn_female'] = df['earn_female'].fillna(0)


# Rearrange the column from the very end of the dataframe to the fourth column
cols = df.columns.tolist()
df = df[cols[0:2] + [cols[-7]] + cols[2:5] + [cols[-5]] + cols[5:27] + [cols[-4]] + [cols[-1]] + [cols[-2]] +[cols[-3]] + [cols[-6]] +cols[27:len(cols)-7]] # cols[-1] takes the last value of cols; cols[0:3] selects columns 0 to 2



#################################################################################################################


#### Data Rearrangement

# Sort the data chronologically
df_chronological = df.sort_values(['ssuid', 'swave', 'srefmon_numerical'], ascending = [1,1,1])

# Select only waves 1 and 2
wave12 = df_chronological[(df_chronological.swave == 1) | (df_chronological.swave == 2)]
wave12_earnings = wave12[(wave12.tfearn != 'NaN') | (wave12.tfearn != 'None or not in universe')]



#################################################################################################################



#### Check cleaning criteria

# missing labor earnings
missing_labor = wave12_earnings[(wave12_earnings.tfearn == "NaN") | (wave12_earnings.tfearn == 'None or not in universe')].size

# cohabiting nonmarried couples
check1 = wave12_earnings[(wave12_earnings.famsize > 1) & (wave12_earnings.spouse == 0)]
check2 = check1[(check1.totearn != check1.earn_male) & (check1.totearn != check1.earn_female)]
number_cohabiting_nonmarried_couples = len(check2.ssuid.unique())

# age
min_age = df.tage.min()
max_age = df.tage.max()



#################################################################################################################



#### Exclusion of households with less than 8 monthly labor earnings in both waves

households = wave12_earnings.groupby('ssuid').count()
target_households = households[households.swave == 8].axes[0].tolist()
percentage_8_month = len(target_households) / len(wave12_earnings.ssuid.unique())




#################################################################################################################



#### Correct for marital changes

final_data = wave12_earnings[wave12_earnings['ssuid'].isin(target_households)]

# Split the dataset
marital_nonchange = final_data[final_data.groupby(['ssuid'])['spouse'].transform('nunique') <= 1]
marital_change = final_data[final_data.groupby(['ssuid'])['spouse'].transform('nunique') > 1]

# Create a list with all of the entries of column 'spouse'
marital_values = marital_change.spouse.tolist()

# Pick every 8th element of the list, starting with element 0
marital_values = marital_values[0::8]

# Duplicate every element 8 times 
marital_values = [x for item in marital_values for x in repeat(item, 8)]

# Replace column 'spouse' in the marital_change dataframe with the elements of marital_values
marital_change['spouse'] = marital_values # Here I get the setting with copy warning

# Combine the splitted dataframes again and sort it
final_data = pd.concat([marital_nonchange, marital_change]) 
final_data = final_data.sort_values(['ssuid', 'swave', 'srefmon_numerical'], ascending = [1,1,1])







#################################################################################################################


#### Exclusion of households that indicate multiple rebate months

multiple_rebates = final_data[final_data.groupby(['ssuid'])['reb_month_numerical'].transform('nunique') > 1]
final_data = final_data[final_data.groupby(['ssuid'])['reb_month_numerical'].transform('nunique') <= 1]


#################################################################################################################



#### Compute the Labor Income Ratio

# Take the mean of every consecutive 8 values
labor_income = np.array(final_data.tfearn)
mean_labor_income = np.mean(labor_income.reshape(-1, 8), axis=1)

# Duplicate every value 8 times and place it behind each other
mean_labor_income = [x for value in mean_labor_income for x in repeat(value, 8)]

# Create a new column and place it behind the tfearn column
final_data['tfearn_mean'] = mean_labor_income

# Compute the relation for every row in the data
labor_income_ratio = labor_income / mean_labor_income
final_data['labor_income_ratio'] = labor_income_ratio

# Rearrange columns in the data
cols = final_data.columns.tolist()
final_data = final_data[cols[0:10] + [cols[-2]] + [cols[-1]] + cols[10:len(cols)-2]]
final_data['labor_income_ratio'] = final_data['labor_income_ratio'].fillna(0)





#################################################################################################################




#### Exclude every household that reported negative labor earnings at least once

neg_income = final_data[final_data.tfearn < 0]
households_neg_income = neg_income.ssuid.unique()
final_data = final_data[~final_data['ssuid'].isin(households_neg_income)]




#################################################################################################################





#### Exclusion of households indicating that they both have and have not received a rebate

help_df = final_data.groupby(['ssuid']).mean()
remain = help_df[(help_df.reb_receipt == 1) | (help_df.reb_receipt == 0)]
final_data = final_data[final_data.ssuid.isin(remain.index)]






#################################################################################################################



#### Create dummies for the year

year_dummy = pd.get_dummies(final_data.year)

year_2008 = year_dummy.iloc[:,0]
year_2009 = year_dummy.iloc[:,1]



#### Create dummies for the months --> April month not in data

month_dummy = pd.get_dummies(final_data.month)

Jan = month_dummy.loc[:,'January']
Feb = month_dummy.loc[:,'February']
Mar = month_dummy.loc[:,'March']

May = month_dummy.loc[:,'May']
Jun = month_dummy.loc[:,'June']
Jul = month_dummy.loc[:,'July']
Aug = month_dummy.loc[:,'August']
Sep = month_dummy.loc[:,'September']
Oct = month_dummy.loc[:,'October']
Nov = month_dummy.loc[:,'November']
Dec = month_dummy.loc[:,'December']


#### Add Dummies as columns to the data

final_data['year_2008'] = year_2008
final_data['year_2009'] = year_2009

final_data['Dummy_January'] = Jan
final_data['Dummy_February'] = Feb
final_data['Dummy_March'] = Mar
final_data['Dummy_May'] = May
final_data['Dummy_June'] = Jun
final_data['Dummy_July'] = Jul
final_data['Dummy_August'] = Aug
final_data['Dummy_September'] = Sep
final_data['Dummy_October'] = Oct
final_data['Dummy_November'] = Nov
final_data['Dummy_December'] = Dec


#################################################################################################################




#### Reference month transformation
final_data = pd.concat([final_data, pd.get_dummies(final_data['srefmon'])], axis=1)
final_data.rename(columns = {'First Reference month': 'first_reference_month', 
                             'Second Reference month': 'second_reference_month', 
                             'Third Reference month': 'third_reference_month', 
                             'Fourth Reference month': 'fourth_reference_month'})




#### wave transformation
final_data = pd.concat([final_data, pd.get_dummies(final_data['swave'])], axis = 1)

final_data = final_data.rename({1: 'Wave_1', 2: 'Wave_2'}, axis=1) 

#################################################################################################################



final_data['rebate_uplifting'] = final_data['reb_size'] * final_data['rebate_payment']
final_data['rebate_uplifting_spend'] = final_data['reb_spend'] * final_data['rebate_payment']
final_data['rebate_uplifting_save'] = final_data['reb_save'] * final_data['rebate_payment']
final_data['rebate_uplifting_debt'] = final_data['reb_debt'] * final_data['rebate_payment']





#################################################################################################################



#### Save the manipulated data as csv file

# Save the dataframe as csv without copying the index in the first column
final_data.to_csv('Cleaned_Data_Household.csv', index=False)




C:\Users\User\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:239: RuntimeWarning: invalid value encountered in true_divide


# Raw Data Input & Data Manipulation -> individual.dta

In [2]:
#### Import Raw Data from individual.dta file

df = pd.read_stata('individual.dta', convert_categoricals = False) # , convert_categoricals = False
feature_names = df.columns
df_shape = df.shape
number_households_df = len(df.ssuid.unique())

#################################################################################################################

#### Data Manipulation

# Reference month transformation
df['srefmon_numerical'] = df['srefmon']


# Observation month transformation
df['month_numerical'] = df['month']


# Month of rebate receipt
def reb_month (row):
   if row['reb4'] == 1 :
      return 'April'
   if row['reb5'] == 1 :
      return 'May'
   if row['reb6'] == 1 :
      return 'June'
   if row['reb7'] == 1 :
      return 'July'
   if row['reb8'] == 1 :
      return 'August'
   if row['reb9'] == 1 :
      return 'September'
   if row['reb10'] == 1 :
      return 'October'
   if row['reb11'] == 1 :
      return 'November'
   if row['reb12'] == 1 :
      return 'December'
   return 'No rebate'

df['reb_month'] = df.apply (lambda row: reb_month(row), axis=1)



def reb_month_numerical (row):
    if row['reb4'] == 1 :
      return 4
    if row['reb5'] == 1 :
      return 5
    if row['reb6'] == 1 :
      return 6
    if row['reb7'] == 1 :
      return 7
    if row['reb8'] == 1 :
      return 8
    if row['reb9'] == 1 :
      return 9
    if row['reb10'] == 1 :
      return 10
    if row['reb11'] == 1 :
      return 11
    if row['reb12'] == 1 :
      return 12
    return 0 

df['reb_month_numerical'] = df.apply (lambda row: reb_month_numerical(row), axis=1)


# Rebate in this month
def rebate_payment (row):
   if row['reb_month_numerical'] == row['month'] :
      return 1
   return 0

df['rebate_payment'] = df.apply (lambda row: rebate_payment(row), axis=1)


# Dummy creation for rebate receipt
df['reb_receipt'] = df.reb4 + df.reb5 + df.reb6 + df.reb7 + df.reb8 + df.reb9 + df.reb10 + df.reb11 + df.reb12
df['reb_size'] = df.amt4 + df.amt5 + df.amt6 + df.amt7 + df.amt8 + df.amt9 + df.amt10 + df.amt11 + df.amt12



#df['earn_male'] = df['earn_male'].fillna(0)
#df['earn_female'] = df['earn_female'].fillna(0)


# Rearrange the column from the very end of the dataframe to the fourth column
cols = df.columns.tolist()
df = df[cols[0:2] + [cols[-7]] + cols[2:5] + [cols[-5]] + cols[5:27] + [cols[-4]] + [cols[-1]] + [cols[-2]] +[cols[-3]] + [cols[-6]] +cols[27:len(cols)-7]] # cols[-1] takes the last value of cols; cols[0:3] selects columns 0 to 2



#################################################################################################################


#### Data Rearrangement

# Sort the data chronologically
df_chronological = df.sort_values(['ssuid', 'swave', 'srefmon'], ascending = [1,1,1])

# Select only waves 1 and 2
wave12 = df_chronological[(df_chronological.swave == 1) | (df_chronological.swave == 2)]
wave12_earnings = wave12[(wave12.tpearn != 'NaN') | (wave12.tpearn != 'None or not in universe')]



#################################################################################################################



#### Check cleaning criteria

# missing labor earnings
missing_labor = wave12_earnings[(wave12_earnings.tpearn == "NaN") | (wave12_earnings.tpearn == 'None or not in universe')].size


#################################################################################################################


#### Exclusion of households with less than 8 monthly labor earnings in both waves

households = wave12_earnings.groupby('ssuid').count()
target_households = households[households.swave >= 8].axes[0].tolist()
percentage_8_month = len(target_households) / len(wave12_earnings.ssuid.unique())



#################################################################################################################

#### For the individual analysis, I don't check for marital changes in the data

#### Correct for marital changes

# Split the dataset
#marital_nonchange = final_data[final_data.groupby(['ssuid'])['spouse'].transform('nunique') <= 1]
#marital_change = final_data[final_data.groupby(['ssuid'])['spouse'].transform('nunique') > 1]

# Create a list with all of the entries of column 'spouse'
#marital_values = marital_change.spouse.tolist()

# Pick every 8th element of the list, starting with element 0
#marital_values = marital_values[0::8]

# Duplicate every element 8 times 
#marital_values = [x for item in marital_values for x in repeat(item, 8)]

# Replace column 'spouse' in the marital_change dataframe with the elements of marital_values
#marital_change['spouse'] = marital_values # Here I get the setting with copy warning

# Combine the splitted dataframes again and sort it
#final_data = pd.concat([marital_nonchange, marital_change]) 




final_data = wave12_earnings[wave12_earnings['ssuid'].isin(target_households)]






#### Exclude every household that reported negative labor earnings at least once


neg_income = final_data[final_data.tpearn < 0]
households_neg_income = neg_income.ssuid.unique()
final_data = final_data[~final_data['ssuid'].isin(households_neg_income)]







#################################################################################################################


#### Exclusion of households that indicate multiple rebate months

multiple_rebates = final_data[final_data.groupby(['ssuid'])['reb_month_numerical'].transform('nunique') > 1]
final_data = final_data[final_data.groupby(['ssuid'])['reb_month_numerical'].transform('nunique') <= 1]


#################################################################################################################





#################################################################################################################

final_data = final_data.sort_values(['ssuid', 'epppnum', 'swave', 'srefmon'], ascending = [1,1,1,1])


final_data['individual_id'] = final_data['ssuid'].astype(str) + final_data['epppnum'].astype(str)

# Rearrange the column
cols = final_data.columns.tolist()
final_data = final_data[[cols[-1]] + cols[0:len(cols)-1]] # cols[-1] takes the last value of cols; cols[0:3] selects columns 0 to 2

#### Compute the Labor Income Ratio



#ids = final_data.ssuid.unique().tolist()
#pers_num = final_data.epppnum.unique().tolist()
#help_columns = final_data.columns.tolist() + list(['tpearn_mean'])

#df = pd.DataFrame(columns = help_columns)


#for i in ids:
#    for j in pers_num:
#    
#        df1 = final_data[(final_data.ssuid == i) & (final_data.epppnum == j)]
#        
#        mean_tpearn = df1.tpearn.mean()
#        
#        df1['tpearn_mean'] = mean_tpearn
#        
#        df = df.append(df1)
#
#        

# Create a new column and place it behind the tfearn column
#final_data = df.copy()


# Compute the relation for every row in the data
#labor_income_ratio = final_data['tpearn'] / final_data['tpearn_mean']
#labor_income_ratio[np.isinf(labor_income_ratio)] = 0
#final_data['labor_income_ratio'] = labor_income_ratio
#final_data['labor_income_ratio'] = final_data['labor_income_ratio'].fillna(0)


new = (final_data.groupby(['individual_id']).count().swave == 8)
target_individuals_id = new[new == True].index
final_data = final_data[final_data.individual_id.isin(target_individuals_id)]

#### Compute the Labor Income Ratio

# Take the mean of every consecutive 8 values
labor_income = np.array(final_data.tpearn)
mean_labor_income = np.mean(labor_income.reshape(-1, 8), axis=1)

# Duplicate every value 8 times and place it behind each other
mean_labor_income = [x for value in mean_labor_income for x in repeat(value, 8)]

# Create a new column and place it behind the tfearn column
final_data['tpearn_mean'] = mean_labor_income

# Compute the relation for every row in the data
labor_income_ratio = labor_income / mean_labor_income
final_data['labor_income_ratio'] = labor_income_ratio
final_data['labor_income_ratio'] = final_data['labor_income_ratio'].fillna(0)



#################################################################################################################



#### Exclusion of households indicating that they both have and have not received a rebate

help_df = final_data.groupby(['ssuid']).mean()
remain = help_df[(help_df.reb_receipt == 1) | (help_df.reb_receipt == 0)]
final_data = final_data[final_data.ssuid.isin(remain.index)]



#################################################################################################################



#### Create dummies for the year

year_dummy = pd.get_dummies(final_data.year)

year_2008 = year_dummy.iloc[:,0]
year_2009 = year_dummy.iloc[:,1]



#### Create dummies for the months --> April month not in data

month_dummy = pd.get_dummies(final_data.month)

Jan = month_dummy.loc[:,1]
Feb = month_dummy.loc[:,2]
Mar = month_dummy.loc[:,3]

May = month_dummy.loc[:,5]
Jun = month_dummy.loc[:,6]
Jul = month_dummy.loc[:,7]
Aug = month_dummy.loc[:,8]
Sep = month_dummy.loc[:,9]
Oct = month_dummy.loc[:,10]
Nov = month_dummy.loc[:,11]
Dec = month_dummy.loc[:,12]


#### Add Dummies as columns to the data

final_data['year_2008'] = year_2008
final_data['year_2009'] = year_2009

final_data['Dummy_January'] = Jan
final_data['Dummy_February'] = Feb
final_data['Dummy_March'] = Mar
final_data['Dummy_May'] = May
final_data['Dummy_June'] = Jun
final_data['Dummy_July'] = Jul
final_data['Dummy_August'] = Aug
final_data['Dummy_September'] = Sep
final_data['Dummy_October'] = Oct
final_data['Dummy_November'] = Nov
final_data['Dummy_December'] = Dec



#################################################################################################################




#### Reference month transformation

ref_dummy = pd.get_dummies(final_data.srefmon)

ref1 = ref_dummy.loc[:,1]
ref2 = ref_dummy.loc[:,2]
ref3 = ref_dummy.loc[:,3]
ref4 = ref_dummy.loc[:,4]

final_data['First Reference month'] = ref1
final_data['Second Reference month'] = ref2
final_data['Third Reference month'] = ref3
final_data['Fourth Reference month'] = ref4


#### wave transformation
final_data = pd.concat([final_data, pd.get_dummies(final_data['swave'])], axis = 1)

final_data = final_data.rename({1: 'Wave_1', 2: 'Wave_2'}, axis=1) 



#################################################################################################################




final_data['rebate_uplifting'] = final_data['reb_size'] * final_data['rebate_payment']
final_data['rebate_uplifting_spend'] = final_data['reb_spend'] * final_data['rebate_payment']
final_data['rebate_uplifting_save'] = final_data['reb_save'] * final_data['rebate_payment']
final_data['rebate_uplifting_debt'] = final_data['reb_debt'] * final_data['rebate_payment']




#################################################################################################################


#### compute the average wage 

final_data['sum of hours worked'] = final_data['ehrsall']*final_data['rwksperm']
final_data['average hourly wage'] = final_data['tpearn'] / final_data['sum of hours worked']

final_data['average hourly wage'][final_data['average hourly wage'] <= 0] = 0













#################################################################################################################


final_data = final_data.sort_values(['ssuid', 'epppnum', 'swave', 'srefmon'], ascending = [1,1,1,1])



#### Save the manipulated data as csv file

# Save the dataframe as csv without copying the index in the first column
final_data.to_csv('Cleaned_Data_Individual.csv', index=False)



C:\Users\User\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:256: RuntimeWarning: invalid value encountered in true_divide
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
